In [1]:
import pandas as pd
import numpy as np

Dataset from kaggle 
[text](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata)


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head(1)

## checking the similar columns to merge the two dataframes

In [ ]:
similar_columns = list(set(movies.columns).intersection(set(credits.columns)))
similar_columns

In [ ]:
# merging the data frame on title

movies = movies.merge(credits, on= similar_columns)
movies.info()

In [ ]:

movies = movies[['movie_id', 'keywords', 'genres','overview', 'vote_average', 'vote_count', 'cast', 'crew', 'title']]
movies.head()

In [ ]:
movies.isnull().sum()

# Data Preprocessing

In [41]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [43]:
import ast

In [45]:
def dataPreprocessor(obj) :
    moive_type = []
    for i in ast.literal_eval(obj):
        moive_type.append(i['name'])
    return moive_type




# taking best 3 actors only
def castPreProcessor(obj) :
    actors = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3 : 
            actors.append(i['name'])
            counter +=1
        else: break
    return actors



# taking the director out of crew 


def fetch_director (obj):
    directors = []
    for i in ast.literal_eval(obj):
        if  i['job'] == 'Director':
            directors.append( i['name'])
            break
    return directors

In [46]:
movies['genres']  = movies['genres'].apply(dataPreprocessor)
movies['keywords']  = movies['keywords'].apply(dataPreprocessor)
movies['cast']  = movies['cast'].apply(castPreProcessor)
movies['crew']  = movies['crew'].apply(fetch_director)

In [47]:
movies['overview'] = movies['overview'].apply(lambda x: x.split() )

In [ ]:
movies.head(1)

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" ", "") for i in x] )
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" ", "") for i in x] )
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" ", "") for i in x] )
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" ", "") for i in x] )
movies.head(1)

In [ ]:
movies['tag'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head(1)

In [ ]:
df = movies[['movie_id', 'title', 'tag']]
df.head()

In [ ]:
df['tag'] = df['tag'].apply(lambda x : " ".join(x))

In [ ]:
df['tag'] = df['tag'].apply(lambda x : x.lower())

# Stemming the text

In [54]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [55]:
def stemmingTags (text):
    stemmed = []
    for i in text.split():
        stemmed.append(ps.stem(i))
    return  " ".join(stemmed)

In [ ]:
df['tag']= df['tag'].apply(stemmingTags)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(df['tag']).toarray()

# distance between different vectors

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend_movie(movie):
    movie_index =df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[1:6]   # first 5 excluding the one whcih is itself
    for i in movies_list:
        print(df.iloc[i[0]].title)
recommend_movie('The Martian')

In [93]:
import pickle as pkl

In [95]:
pkl.dump(df.to_dict(), open('movies.pkl', 'wb'))

In [97]:
pkl.dump(similarity,open('similarity.pkl', 'wb'))